In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


In [3]:
df = pd.read_csv("data.csv", sep=";", decimal=",")
df = df.rename(columns={
    "Column1": "Date",
    "Column2": "SPX",
    "Column3": "S5SFTW",
    "Column4": "S5PHRM",
    "Column5": "S5CPGS",
    "Column6": "S5ENRSX",
    "Column7": "S5FDBT",
    "Column8": "S5TECH",
    "Column9": "S5RETL",
    "Column10": "S5BANKX",
    "Column11": "S5HCES",
    "Column12": "S5DIVF",
    "Column13": "S5UTILX",
    "Column14": "S5MEDA",
    "Column15": "S5REAL",
    "Column16": "S5TELSX",
    "Column17": "S5MATRX",
    "Column18": "S5INSU",
    "Column19": "S5FDSR",
    "Column20": "S5HOUS",
    "Column21": "S5SSEQX",
    "Column22": "S5TRAN",
    "Column23": "S5HOTR",
    "Column24": "S5CODU",
    "Column25": "S5AUCO",
    "Column26": "S5COMS",
})
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")
df["MonthIndex"] = df["Date"].dt.to_period("M")

#----------
#PARAMETERS
#----------
fees_rate=0.0002
# const
df_length = df.shape[1] - 2  # bcs of date and spx
last_rebalance = df.loc[0, "Date"]  # première date
month_count = 0

#Return a percentage between a and b
def evolution(v1, v2):
    return round((v2 - v1) / v1 * 100, 5)

#-------------------
#SORTER BETWEEN PERF
#-------------------

def DecilSorter(start, hist, proportion=3):
    bestperformer = []
    worstperformer = []
    performerc = []

    for i in range(2, df_length-1):
      performerc.append((evolution(float(df.iloc[start, i]), float(df.iloc[start + hist, i])), i))

    performerc.sort(reverse=True)
    offset=proportion
    for i in range(proportion):
        bestperformer.append(performerc[i][1])

    for i in range(len(performerc) - offset - proportion, len(performerc) - offset):
        worstperformer.append(performerc[i][1])


    return bestperformer, worstperformer


#--------------
#ALGO TITMANJEG
#--------------

def JeegadesshTitmanStrat(start=0, hold=6, hist=6, proportion=3):
    #new dataframe for stock quantity
    Indexcolumns = ["Date","SPX","S5SFTW","S5PHRM","S5CPGS","S5ENRSX","S5FDBT","S5TECH","S5RETL","S5BANKX","S5HCES","S5DIVF","S5UTILX","S5MEDA","S5REAL","S5TELSX","S5MATRX","S5INSU","S5FDSR","S5HOUS","S5SSEQX","S5TRAN","S5HOTR","S5CODU","S5AUCO","S5COMS","Money"]
    Tickercolumns=["S5SFTW","S5PHRM","S5CPGS","S5ENRSX","S5FDBT","S5TECH","S5RETL","S5BANKX","S5HCES","S5DIVF","S5UTILX","S5MEDA","S5REAL","S5TELSX","S5MATRX","S5INSU","S5FDSR","S5HOUS","S5SSEQX","S5TRAN","S5HOTR","S5CODU","S5AUCO","S5COMS"]
    StockQty = df.copy()
    StockQty.drop(columns="MonthIndex", inplace=True)
    StockQty.loc[:, :] = 0

    #starting data
    MoneyAtStart = 10000000
    month_count=0
    CurrentValue=MoneyAtStart

    #first ligne
    StockQty.loc[0, "Money"] = MoneyAtStart
    StockQty.loc[0, "SPX"] = df.iloc[0, 1]
    StockQty.loc[0, "Date"] = df.iloc[0, 0]

    #start of the algorithm

    for i in range(1,df.shape[0]):
      StockQty.iloc[i,0]=df.iloc[i,0]
      StockQty.iloc[i,1]=df.iloc[i,1]
      fees=0

      if df.loc[i, "Date"].month != df.loc[i-1, "Date"].month:
        month_count += 1


    # Si on atteint la période voulue
      if (i>= hist and month_count % hold == 0 and df.loc[i, "Date"].month != df.loc[i - 1, "Date"].month):
        #print(f"🔁 Rebalancement déclenché à la date : {df.loc[i, 'Date'].date()}")
        bestperformer, worstperformer = DecilSorter(i-hist, hist, proportion)


        for bestperf in bestperformer:
          StockQty.iloc[i,bestperf]=round((CurrentValue/(len(bestperformer)*2))/float(df.iloc[i,bestperf]),2)


        for worstperf in worstperformer:
          StockQty.iloc[i,worstperf]=round(-1*(CurrentValue/(len(worstperformer)*2))/float(df.iloc[i,worstperf]),2)

        for stocks in range(2,StockQty.shape[1]-1):
          if StockQty.iloc[i,stocks]!=StockQty.iloc[i-1,stocks]:
            fees+=abs(StockQty.iloc[i,stocks]-StockQty.iloc[i-1,stocks])*df.iloc[i,stocks]*fees_rate

      else :
        for stocks in range(2,StockQty.shape[1]-1):
          StockQty.iloc[i,stocks]=StockQty.iloc[i-1,stocks] #same qty
      #value of pf

      GainOrLoss = 0
      for stocks in range(2, StockQty.shape[1]-1):
          qty = StockQty.iloc[i, stocks]
          if qty != 0.0:
            price_now = df.iloc[i, stocks]
            price_prev = df.iloc[i - 1, stocks]
            GainOrLoss += qty * (price_now - price_prev)


      CurrentValue+=GainOrLoss-fees
      StockQty.iloc[i,-1]=CurrentValue

    return StockQty

results = JeegadesshTitmanStrat(0, 3, 9*22 , 3)


#---- GRAPH

"""
money_norm =(results["Money"]/10000000*100)-100
spx_norm=(results["SPX"]/results["SPX"].iloc[0]* 100)-100

fix = px.line(x=results["Date"], y=[money_norm, spx_norm],labels={"value":"Évolution en %", "variable":"Série"},title="Comparaison des évolutions en %")
fix.update_layout(hovermode="x unified")

fix.show()
"""

#-------- ANNUALISATION

AnnualizedDf=results[["Date","SPX","Money"]]
AnnualizedDf['Date'] = pd.to_datetime(AnnualizedDf['Date'])
AnnualizedDf['Year'] = AnnualizedDf['Date'].dt.year



YearList=AnnualizedDf["Year"].unique()
SPXAnnualized=pd.DataFrame(columns=YearList)
StratAnnualized=pd.DataFrame(columns=YearList)



for year in YearList:
  compteurPerYear=0
  for i in AnnualizedDf.index:
    if AnnualizedDf.loc[i,"Year"]==year:
      if compteurPerYear==0:
        SPXAnnualized.loc[compteurPerYear,year]=AnnualizedDf.loc[i,"SPX"]
        StratAnnualized.loc[compteurPerYear,year]=AnnualizedDf.loc[i,"Money"]
      else :
        SPXAnnualized.loc[compteurPerYear,year]=AnnualizedDf.loc[i,"SPX"]/SPXAnnualized.loc[0,year]*100-100
        StratAnnualized.loc[compteurPerYear,year]=AnnualizedDf.loc[i,"Money"]/StratAnnualized.loc[0,year]*100-100
      compteurPerYear+=1

for year in YearList:
  SPXAnnualized.loc[0,year]=SPXAnnualized.loc[0,year]/SPXAnnualized.loc[0,year]*100-100
  StratAnnualized.loc[0,year]=StratAnnualized.loc[0,year]/StratAnnualized.loc[0,year]*100-100



SPXAvg=[]
StratAvg=[]
for i in SPXAnnualized.index:
  sumSPX=0
  sumStrat=0
  for year in SPXAnnualized.columns:
    sumSPX+=SPXAnnualized.loc[i,year]
    sumStrat+=StratAnnualized.loc[i,year]
  SPXAvg.append(sumSPX/len(YearList))
  StratAvg.append(sumStrat/len(YearList))

#------- GRAPH / YEAR
"""
fig = px.line(SPXAnnualized, x=SPXAnnualized.index, y=SPXAnnualized.columns,labels={"value":"Returns in % of the SPX", "variable":"Years"},title="Comparaison of the returns on differents years | SPX")
fig.update_xaxes(tickmode='linear',  dtick=1,)
fig.show()


fig = px.line(StratAnnualized, x=StratAnnualized.index, y=StratAnnualized.columns,labels={"value":"Returns in % of the Start", "variable":"Years"},title="Comparaison of the returns on differents years | Strat")
fig.update_xaxes(tickmode='linear',  dtick=1,)
fig.show()
"""

SPXAnnualized=SPXAnnualized.drop(columns=2024) #too much nan
StratAnnualized=StratAnnualized.drop(columns=2024)

SPXAvg=[]
StratAVG=[]

for i in SPXAnnualized.index:
  sumSPX=0
  sumStrat=0
  for year in SPXAnnualized.columns:
    sumSPX+=SPXAnnualized.loc[i,year]
    sumStrat+=StratAnnualized.loc[i,year]
  SPXAvg.append(sumSPX/len(YearList))
  StratAVG.append(sumStrat/len(YearList))

dff = pd.DataFrame({"Index": (range(len(SPXAvg))),"SPX": SPXAvg,"Strat": StratAVG})


fig = px.line(dff, x="Index", y=["SPX","Strat"])
fig.show()


#---------- VOL


VolSPX = df[["Date", "SPX"]].copy()
VolSPX["Date"] = pd.to_datetime(VolSPX["Date"], dayfirst=True)
VolSPX["Year"] = VolSPX["Date"].dt.year
VolSPX["Return"] = np.log(VolSPX["SPX"] / VolSPX["SPX"].shift(1))
NumberofDayInYear=VolSPX.groupby("Year")["SPX"].count()
VolSPX = (VolSPX.groupby("Year")["Return"].std())
VolSPX = VolSPX * np.sqrt(NumberofDayInYear)


VolStrat=AnnualizedDf[["Money","Year"]].copy()
VolStrat["Money"]=VolStrat["Money"].astype(float)
VolStrat["Return"] = np.log(VolStrat["Money"] / VolStrat["Money"].shift(1))
NumberofDayInStrat=VolStrat.groupby("Year")["Money"].count()
VolStrat = (VolStrat.groupby("Year")["Return"].std())
VolStrat = VolStrat * np.sqrt(NumberofDayInStrat)

Vol=pd.concat([VolSPX,VolStrat],axis=1)
Vol.columns=["SPX","Strat"]


fig = px.line(Vol, x=Vol.index, y=Vol.columns,labels={"value":"Vol",},title="Comparaison of vol : SPX / Strat")
fig.update_xaxes(tickmode='linear',  dtick=1,)
fig.show()


C:\Users\mbard\AppData\Local\Temp\ipykernel_32272\3385403260.py:80: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  StockQty.loc[:, :] = 0
C:\Users\mbard\AppData\Local\Temp\ipykernel_32272\3385403260.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2002-01-01 00:00:00' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  StockQty.loc[0, "Date"] = df.iloc[0, 0]
C:\Users\mbard\AppData\Local\Temp\ipykernel_32272\3385403260.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [4]:
results

,Date,SPX,S5SFTW,S5PHRM,S5CPGS,S5ENRSX,S5FDBT,S5TECH,S5RETL,S5BANKX,...,S5INSU,S5FDSR,S5HOUS,S5SSEQX,S5TRAN,S5HOTR,S5CODU,S5AUCO,S5COMS,Money
0,2002-01-01 00:00:00,1148.08,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.000000e+07
1,2002-01-02 00:00:00,1154.67,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.000000e+07
2,2002-01-03 00:00:00,1165.27,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.000000e+07
3,2002-01-04 00:00:00,1172.51,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.000000e+07
4,2002-01-07 00:00:00,1164.89,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.000000e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5778,2024-02-23 00:00:00,5088.80,0.0,-1809.24,0.0,-3606.44,0.0,0.0,593.58,0.0,...,0.0,0.0,0.0,703.86,0.0,0.0,0.0,0.0,0.0,1.466253e+07
5779,2024-02-26 00:00:00,5069.53,0.0,-1809.24,0.0,-3606.44,0.0,0.0,593.58,0.0,...,0.0,0.0,0.0,703.86,0.0,0.0,0.0,0.0,0.0,1.466167e+07
5780,2024-02-27 00:00:00,5078.18,0.0,-1809.24,0.0,-3606.44,0.0,0.0,593.58,0.0,...,0.0,0.0,0.0,703.86,0.0,0.0,0.0,0.0,0.0,1.467943e+07
5781,2024-02-28 00:00:00,5069.76,0.0,-1809.24,0.0,-3606.44,0.0,0.0,593.58,0.0,...,0.0,0.0,0.0,703.86,0.0,0.0,0.0,0.0,0.0,1.462293e+07


In [33]:
def GetReturn(df,date,lookback=180):
    date=pd.to_datetime(date)
    if date not in df["Date"].values:#add breaker if windows not in df
        raise ValueError("Date not in dataframe")
    returns_df = df[["Date","S5SFTW","S5PHRM","S5CPGS","S5ENRSX","S5FDBT","S5TECH","S5RETL","S5BANKX","S5HCES","S5DIVF","S5UTILX","S5MEDA","S5REAL","S5TELSX","S5MATRX","S5INSU","S5FDSR","S5HOUS","S5SSEQX","S5TRAN","S5HOTR","S5CODU","S5AUCO","S5COMS"]].copy()

    date_list=returns_df.drop(columns="Date")
    date_index = returns_df.index[returns_df["Date"] == date][0]
    returns_df=returns_df[(returns_df.index<=date_index) & (returns_df.index>=date_index-lookback) ]
    returns_df.drop(columns="Date",inplace=True)

    returns_df = np.log(returns_df/ returns_df.shift(1))
    returns_df.dropna(inplace=True)
    #print(returns_df.std().mean()) #verification if std is around 1% daily

    return returns_df

In [21]:
def GetSigma(df,date,lookback=180):
    returns_df=GetReturn(df,date,lookback=lookback)
    #covariance matric from returns_df
    sigma_windowed=returns_df.cov()

    return sigma_windowed

0.012426235198907357


,S5SFTW,S5PHRM,S5CPGS,S5ENRSX,S5FDBT,S5TECH,S5RETL,S5BANKX,S5HCES,S5DIVF,...,S5MATRX,S5INSU,S5FDSR,S5HOUS,S5SSEQX,S5TRAN,S5HOTR,S5CODU,S5AUCO,S5COMS
S5SFTW,0.000162,0.000051,0.000073,3.510724e-05,0.000017,0.000126,0.000122,0.000063,0.000078,0.000081,...,0.000072,0.000062,0.000053,0.000027,0.000191,0.000074,0.000090,0.000097,0.000162,0.000099
S5PHRM,0.000051,0.000072,0.000028,1.342349e-06,0.000024,0.000050,0.000035,0.000023,0.000047,0.000033,...,0.000035,0.000031,0.000030,0.000031,0.000056,0.000026,0.000012,0.000033,0.000025,0.000043
S5CPGS,0.000073,0.000028,0.000108,1.021936e-04,0.000035,0.000070,0.000069,0.000108,0.000055,0.000093,...,0.000089,0.000086,0.000045,0.000031,0.000107,0.000087,0.000090,0.000095,0.000097,0.000077
S5ENRSX,0.000035,0.000001,0.000102,2.978980e-04,0.000033,0.000038,0.000046,0.000166,0.000025,0.000111,...,0.000088,0.000097,0.000020,0.000006,0.000066,0.000082,0.000098,0.000069,0.000100,0.000039
S5FDBT,0.000017,0.000024,0.000035,3.266564e-05,0.000058,0.000023,0.000009,0.000042,0.000031,0.000037,...,0.000039,0.000043,0.000035,0.000053,0.000003,0.000036,0.000032,0.000028,-0.000025,0.000032
S5TECH,0.000126,0.000050,0.000070,3.753779e-05,0.000023,0.000174,0.000116,0.000071,0.000076,0.000075,...,0.000070,0.000064,0.000058,0.000029,0.000181,0.000072,0.000088,0.000096,0.000171,0.000080
S5RETL,0.000122,0.000035,0.000069,4.649615e-05,0.000009,0.000116,0.000193,0.000071,0.000056,0.000071,...,0.000063,0.000056,0.000052,0.000015,0.000183,0.000068,0.000085,0.000100,0.000153,0.000079
S5BANKX,0.000063,0.000023,0.000108,1.662869e-04,0.000042,0.000071,0.000071,0.000229,0.000052,0.000146,...,0.000107,0.000133,0.000036,0.000031,0.000110,0.000109,0.000121,0.000094,0.000147,0.000060
S5HCES,0.000078,0.000047,0.000055,2.465918e-05,0.000031,0.000076,0.000056,0.000052,0.000085,0.000060,...,0.000057,0.000055,0.000039,0.000036,0.000092,0.000052,0.000057,0.000065,0.000071,0.000064
S5DIVF,0.000081,0.000033,0.000093,1.111886e-04,0.000037,0.000075,0.000071,0.000146,0.000060,0.000119,...,0.000091,0.000104,0.000039,0.000035,0.000115,0.000091,0.000097,0.000090,0.000104,0.000074


In [44]:
def GetLambda(df,date,lookback=180):
    returns=GetReturn(df,date,lookback=lookback)
    returns=returns+1

    avg_return=returns.prod()-1
    weight_vector=GetWeight(df=0,date=0)
    lambda_value=(returns@np.transpose(weight_vector))-#calcul rf periode/sigma




GetLambda(df,"2020-05-11",lookback=180)






S5SFTW     0.088337
S5PHRM     0.135469
S5CPGS    -0.236393
S5ENRSX   -0.398463
S5FDBT    -0.107411
S5TECH     0.212812
S5RETL     0.099774
S5BANKX   -0.345522
S5HCES     0.012264
S5DIVF    -0.152444
S5UTILX   -0.159297
S5MEDA     0.033806
S5REAL    -0.215033
S5TELSX   -0.115534
S5MATRX   -0.134065
S5INSU    -0.269629
S5FDSR    -0.006119
S5HOUS    -0.078166
S5SSEQX    0.098474
S5TRAN    -0.223058
S5HOTR    -0.314135
S5CODU    -0.163164
S5AUCO    -0.423080
S5COMS    -0.111757
dtype: float64


In [ ]:
def GetWeight(df,date):
    #for the moment we will use the equal weight
    weight_vector=np.array((24,1))
    for i in range(24):
        weight_vector[i]=1/24

    return weight_vector


In [ ]:
def BlackAndLittermanModel(backtestStartDate, rebalancingFrequency, lookbackPeriod, df):
    #implement the full backtest of the black and litterman model

    #---------
    #PARAMETERS
    #---------

    free_asset=0 #proportion of risk free asset allocated in the benchmark
    taux=0.01
    RiskAversion=1

    #--------
    #RiskFree
    #--------
    rf=0.046 #risk free rate

    #to be modified later
    for date in range(df.index[df["Date"] == backtestStartDate][0], 10000000,rebalancingFrequency):
        if date+rebalancingFrequency > df.index[-1]:
            break #break if we exceed the df length


        current_date=df.iloc[date,0]

        SIGMA=GetSigma(df,current_date)
        WEIGHT=GetWeight(df,current_date)












    #-------
    #MARKET
    #-------




    #---------
    #VIEWS
    #---------

















    pass